In [1]:
%matplotlib widget
from ipywidgets import FloatSlider, Output, interact
import matplotlib.pyplot as plt
import numpy as np


In [2]:
filename = 'training1.csv'
_, time, distances, velocity_command, ir1_data, ir2_data, ir3_data, ir4_data, sonar1_data, sonar2_data = np.loadtxt(filename, delimiter=',', skiprows=1).T
N = len(time)
dt = time[1] - time[0]

In [5]:
import matplotlib
%matplotlib widget
@interact(
    a_1 = FloatSlider(min=0, max=1, step=0.01, value=0.5),
    a_2 = FloatSlider(min=0, max=1, step=0.01, value=0.3),
    a_3 = FloatSlider(min=0, max=1, step=0.01, value=0.015),
    a_4 = FloatSlider(min=0, max=1, step=0.01, value=1),
    b_1 = FloatSlider(min=0.9, max=1.5, step=0.01, value=1.1),
    b_2 = FloatSlider(min=0.5, max=1, step=0.01, value=0.7),
)
def parameters(a_1, a_2, a_3, a_4, b_1, b_2):
    def get_alpha(velocity_command: float, last_velocity_command: float, last_predicted_speed: float):
        if abs(velocity_command) * b_1 < abs(last_predicted_speed):
            return a_1
        elif abs(last_predicted_speed / velocity_command) < b_2:
            return a_2
        else:
            return a_3
    
    fig, axs = plt.subplots()
    actual_speeds = np.zeros(N)
    for k in range(1, N):
        actual_speeds[k] = (distances[k] - distances[k-1]) / dt
    kernel = [0.05] * 20
    actual_speeds_smooth = np.convolve(actual_speeds, kernel, mode='same')
    predicted_speeds = np.zeros(N)
    for k in range(1, N):
        alpha = get_alpha(velocity_command[k], velocity_command[k-1], predicted_speeds[k-1])
        # alpha = 0.015 if abs(velocity_command[i]) > 0.07 and not abs(velocity_command[i] - predicted_speeds[i-1]) > 0.12 else 0.1
        predicted_speeds[k] = velocity_command[k] * alpha + (1-alpha) * predicted_speeds[k-1]
    axs.plot(time, velocity_command, label='commanded_speed')
    axs.plot(time, actual_speeds, '.', alpha=0.2, label='actual_speed')
    axs.plot(time, actual_speeds_smooth, label='actual_speed_smooth')
    axs.plot(time, predicted_speeds, label='predicted_speed')
#     axs.set_xlabel('Time (t)')
#     axs.set_ylabel('Speed (m/s)')
#     axs.set_title('Predicting the speed with a recurrsive filter')
#     axs.legend()
#     plt.show()

interactive(children=(FloatSlider(value=0.5, description='a_1', max=1.0, step=0.01), FloatSlider(value=0.3, de…

In [5]:
@interact(x=True, y=1.0)
def g(x, y):
    return (x, y)

interactive(children=(Checkbox(value=True, description='x'), FloatSlider(value=1.0, description='y', max=3.0, …

In [9]:
# set up plot
fig, ax = plt.subplots(figsize=(6, 4))
ax.set_ylim([-4, 4])
ax.grid(True)
 
# generate x values
x = np.linspace(0, 2 * np.pi, 100)
 
 
def my_sine(x, w, amp, phi):
    """
    Return a sine for x with angular frequeny w and amplitude amp.
    """
    return amp*np.sin(w * (x-phi))
 
 
@interact(w=(0, 10, 1), amp=(0, 4, .1), phi=(0, 2*np.pi+0.01, 0.01))
def update(w = 1.0, amp=1, phi=0):
    """Remove old lines from plot and plot new one"""
    [l.remove() for l in ax.lines]
    ax.plot(x, my_sine(x, w, amp, phi), color='C0')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=1, description='w', max=10), FloatSlider(value=1.0, description='amp', m…